<a href="https://colab.research.google.com/github/FarrelAD/Hology-8-2025-Data-Mining-PRIVATE/blob/dev%2Ffarrel/notebooks/farrel/nb_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Density FCN for Crowd Counting

This notebook implements a Density Fully Convolutional Network for crowd counting tasks.

**Features:**
- Custom FCN architecture with backbone feature extractor
- Density map generation with Gaussian filtering
- Data augmentation and preprocessing
- Caching mechanism for density maps
- Model evaluation and visualization

# Import Libraries

In [ ]:
# Import Required Libraries and Setup
import os
import json
import sys
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, maximum_filter
from typing import Optional, Tuple, Dict, Any, Union, List
import zipfile

import torch
from torch import Tensor
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image

print("✅ All libraries imported successfully!")

# Detect environment
def detect_environment() -> str:
    """Detect if running in Colab, Kaggle, or local environment"""
    if 'google.colab' in sys.modules:
        return 'colab'
    elif 'kaggle_secrets' in sys.modules or os.environ.get('KAGGLE_KERNEL_RUN_TYPE'):
        return 'kaggle'
    else:
        return 'local'

ENV = detect_environment()
print(f"🔍 Detected environment: {ENV.upper()}")

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔧 Using device: {device}")

if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

# Dataset Download and Setup

In [ ]:
# Environment-specific dataset setup
def setup_dataset_paths(env: str) -> dict[str, str]:
    """Setup dataset paths based on environment"""
    
    if env == 'colab':
        # Google Colab paths
        dataset_name = "penyisihan-hology-8-0-2025-data-mining"
        drive_path = "/content/drive/MyDrive/PROJECTS/Cognivio/Percobaan Hology 8 2025/dataset"
        local_path = "/content/dataset"
        
        # Mount Google Drive
        from google.colab import drive
        drive.mount('/content/drive')
        
        # Setup Kaggle credentials
        if not os.path.exists("/root/.kaggle/kaggle.json"):
            print("📥 Setting up Kaggle credentials...")
            from google.colab import files
            uploaded = files.upload()
            
            for fn in uploaded.keys():
                print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')
            
            !mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
        
        # Download and setup dataset
        if not os.path.exists(local_path):
            print("📥 Setting up dataset in Colab...")
            
            # Create directories
            os.makedirs(drive_path, exist_ok=True)
            
            zip_path = f"/content/{dataset_name}.zip"
            
            # Download if not exists
            if not os.path.exists(zip_path):
                print("Dataset not found locally, downloading...")
                !pip install -q kaggle
                !kaggle competitions download -c {dataset_name} -p /content
            else:
                print("Dataset already exists, skipping download.")
            
            # Extract to Google Drive (for backup)
            if not os.listdir(drive_path):
                print("Extracting dataset to Google Drive...")
                with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    zip_ref.extractall(drive_path)
                print(f"Dataset extracted to: {drive_path}")
            else:
                print(f"Dataset already extracted at: {drive_path}")
            
            # Copy to local storage for faster access
            print("Copying dataset to Colab local storage (/content)...")
            !cp -r "{drive_path}" "{local_path}"
            print(f"✅ Dataset copied to {local_path}")
        
        return {
            'img_dir': f"{local_path}/train/images",
            'label_dir': f"{local_path}/train/labels",
            'test_dir': f"{local_path}/test/images",
            'save_dir': "/content/drive/MyDrive/PROJECTS/Cognivio/models"
        }
    
    elif env == 'kaggle':
        # Kaggle paths
        return {
            'img_dir': "/kaggle/input/penyisihan-hology-8-0-2025-data-mining/train/images",
            'label_dir': "/kaggle/input/penyisihan-hology-8-0-2025-data-mining/train/labels",
            'test_dir': "/kaggle/input/penyisihan-hology-8-0-2025-data-mining/test/images",
            'save_dir': "/kaggle/working"
        }
    
    else:  # local
        # Local paths - modify these for your setup
        base_path = "../../data"  # Change this to your local dataset path
        return {
            'img_dir': f"{base_path}/train/images",
            'label_dir': f"{base_path}/train/labels",
            'test_dir': f"{base_path}/test/images",
            'save_dir': "models"
        }

# Setup paths
paths = setup_dataset_paths(ENV)
print(f"📁 Dataset paths configured for {ENV}:")
for key, path in paths.items():
    exists = "✅" if os.path.exists(path) else "⚠️"
    print(f"   {key}: {path} {exists}")

# Create save directory
os.makedirs(paths['save_dir'], exist_ok=True)

# Training Configuration

In [ ]:
# Training Configuration
config = {
    # Data paths
    'img_dir': paths['img_dir'],
    'label_dir': paths['label_dir'],
    'test_dir': paths['test_dir'],
    'save_dir': paths['save_dir'],
    
    # Model parameters
    'img_size': 224,
    'downscale_factor': 16,
    'sigma': 4.0,
    
    # Training parameters
    'batch_size': 4,
    'epochs': 100,
    'lr': 0.0001,
    'max_samples': None,  # Use all available samples
    
    # Optimization
    'num_workers': 4,
    'pin_memory': True,
    
    # Data processing
    'cache_density': True,
    'return_meta': False,
    
    # Model saving
    'save_path': os.path.join(paths['save_dir'], 'density_fcn_model.pth'),
}

print("📋 Configuration loaded:")
for key, value in config.items():
    print(f"  {key}: {value}")

print(f"🔧 Using device: {device}")

# Dataset Implementation

In [ ]:
class CrowdCountingDataset(Dataset):
    """Custom dataset for crowd counting with density map generation.
    
    Supports caching of density maps for faster training and various
    data augmentation options.
    """
    
    def __init__(
        self,
        image_dir: str,
        label_dir: str,
        max_samples: Optional[int] = None,
        img_size: int = 224,
        downscale_factor: int = 16,
        sigma: float = 4.0,
        transform: Optional[Any] = None,
        return_meta: bool = False,
        cache_density: bool = True,
    ) -> None:
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.img_size = img_size
        self.downscale_factor = downscale_factor
        self.sigma = sigma
        self.transform = transform
        self.return_meta = return_meta
        self.cache_density = cache_density

        # Collect image files
        all_images = sorted([f for f in os.listdir(image_dir) if f.endswith(".jpg")])
        if max_samples:
            self.image_files = all_images[:max_samples]
        else:
            self.image_files = all_images

        # Pre-load label jsons into memory (fast lookup)
        self.labels = self._load_labels()

        print(f"📦 Using {len(self.image_files)} images for dataset.")

    def __len__(self) -> int:
        return len(self.image_files)

    def __getitem__(
        self,
        idx: int
    ) -> Union[Tuple[torch.Tensor, torch.Tensor], Dict[str, Any]]:
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_dir, img_name)

        # ---- Load and resize image ----
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        orig_h, orig_w = image.shape[:2]
        image = cv2.resize(image, (self.img_size, self.img_size))
        image_tensor = torch.from_numpy(image).permute(2, 0, 1).float() / 255.0

        if self.transform:
            image_tensor = self.transform(image_tensor)

        # ---- Get density map (cached or computed) ----
        density_map = self._get_density_map(img_name, orig_w, orig_h)
        density_map_tensor = torch.from_numpy(density_map).unsqueeze(0)

        # Compute crowd count
        count = float(density_map_tensor.sum())

        if self.return_meta:
            return {
                "image": image_tensor,
                "density": density_map_tensor,
                "count": count,
                "name": img_name,
                "orig_size": (orig_h, orig_w),
            }
        else:
            return image_tensor, density_map_tensor

    def _load_labels(
        self
    ) -> Dict[str, Dict[str, Any]]:
        """Load labels from JSON files into memory."""
        labels = {}
        for img_file in self.image_files:
            label_file = img_file.replace(".jpg", ".json")
            label_path = os.path.join(self.label_dir, label_file)
            if os.path.exists(label_path):
                with open(label_path, "r") as f:
                    labels[img_file] = json.load(f)
        return labels

    def _get_density_map(
        self, 
        img_name: str, 
        orig_w: int, 
        orig_h: int
    ) -> np.ndarray:
        """Generate or load precomputed density map."""
        output_size = self.img_size // self.downscale_factor
        cache_path = os.path.join(self.label_dir, img_name.replace(".jpg", ".npy"))

        # If cached density exists, just load
        if self.cache_density and os.path.exists(cache_path):
            return np.load(cache_path)

        # Otherwise, compute density map
        density_map = np.zeros((output_size, output_size), dtype=np.float32)
        label_data = self.labels.get(img_name)

        if label_data and label_data.get("human_num", 0) > 0:
            raw_points = label_data["points"]

            points = np.array([[p["x"], p["y"]] for p in raw_points], dtype=np.float32)

            for x, y in points:
                scaled_x = (x / orig_w) * output_size
                scaled_y = (y / orig_h) * output_size
                ix, iy = int(scaled_x), int(scaled_y)
                if 0 <= ix < output_size and 0 <= iy < output_size:
                    density_map[iy, ix] += 1.0

        # Apply Gaussian filter
        density_map = gaussian_filter(density_map, sigma=self.sigma / self.downscale_factor)

        # Save cache for future use
        if self.cache_density:
            np.save(cache_path, density_map)

        return density_map

print("✅ CrowdCountingDataset class implemented!")

# Model Architecture

In [ ]:
class DensityFCN(nn.Module):
    """Fully Convolutional Network for density estimation.
    
    Features a backbone feature extractor followed by a density prediction head.
    """
    
    def __init__(self) -> None:
        super(DensityFCN, self).__init__()
        # Backbone: Feature extractor
        self.backbone = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1), 
            nn.ReLU(),
            nn.MaxPool2d(2),  # 224 -> 112
            nn.Conv2d(16, 32, 3, padding=1), 
            nn.ReLU(),
            nn.MaxPool2d(2),  # 112 -> 56
            nn.Conv2d(32, 64, 3, padding=1), 
            nn.ReLU(),
            nn.MaxPool2d(2),  # 56 -> 28
            nn.Conv2d(64, 128, 3, padding=1), 
            nn.ReLU(),
            nn.MaxPool2d(2),  # 28 -> 14
        )
        # Head: Density map predictor
        self.head = nn.Conv2d(128, 1, 1) # 1x1 conv to produce 1-channel map

    def forward(self, x: Tensor) -> Tensor:
        x = self.backbone(x)
        x = self.head(x)
        return x

print("✅ DensityFCN model architecture implemented!")

# Test model instantiation
model_test = DensityFCN().to(device)
print(f"📊 Model parameters: {sum(p.numel() for p in model_test.parameters()):,}")
print("Model architecture:")
print(model_test)

del model_test

# Data Loading and Preparation

In [ ]:
# Check if data directories exist
img_dir = config['img_dir']
label_dir = config['label_dir']

print(f"🔍 Checking data directories...")
print(f"   Images: {img_dir} -> {'✅ Exists' if os.path.exists(img_dir) else '❌ Not found'}")
print(f"   Labels: {label_dir} -> {'✅ Exists' if os.path.exists(label_dir) else '❌ Not found'}")

if not os.path.exists(img_dir):
    raise Exception("❌ Image directory not found. Please check the path and try again.")

# Create dataset
train_dataset = CrowdCountingDataset(
    image_dir=config['img_dir'],
    label_dir=config['label_dir'],
    img_size=config['img_size'],
    downscale_factor=config['downscale_factor'],
    sigma=config['sigma'],
    cache_density=config['cache_density'],
    max_samples=config['max_samples'],
)

# Create data loader
train_loader = DataLoader(
    train_dataset,
    batch_size=config['batch_size'],
    shuffle=True,
    num_workers=config['num_workers'],
    pin_memory=config['pin_memory'],
)

print(f"🚀 Data loader created!")
print(f"   Train batches: {len(train_loader)}")

# Training

## Training Setup

In [ ]:
# Initialize model and training components
print("🏗️  Initializing model and training components...")

# Create model
model = DensityFCN().to(device)
print(f"📊 Model parameters: {sum(p.numel() for p in model.parameters()):,}")

# Loss and Optimizer
criterion = nn.MSELoss() # Pixel-wise MSE for density map regression
optimizer = optim.Adam(model.parameters(), lr=config['lr'])

print("✅ Setup complete! Starting training...")
print("="*60)

## Training Loop

In [ ]:
# Main Training Loop
training_losses = []

print("🚀 Starting training...")
model.train()

for epoch in range(config['epochs']):
    epoch_loss = 0.0
    
    for batch_idx, (images, density_maps) in enumerate(train_loader):
        images = images.to(device)
        density_maps = density_maps.to(device)

        # Forward pass
        optimizer.zero_grad()
        pred_density_maps = model(images)

        # Calculate loss
        loss = criterion(pred_density_maps, density_maps)

        # Backward pass
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(train_loader)
    training_losses.append(avg_loss)

    if (epoch + 1) % 5 == 0 or epoch == 0:
        print(f'📈 Epoch {epoch+1}/{config["epochs"]}, Loss: {avg_loss:.6f}')

print("\n" + "="*60)
print("🎉 Training completed!")

## Training Visualization

In [ ]:
# Plot training loss
plt.figure(figsize=(10, 5))
plt.plot(training_losses, 'b-', linewidth=2)
plt.title('Training Loss Over Time')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.grid(True, alpha=0.3)
plt.show()

print("📈 Training Statistics:")
print(f"   Final Loss: {training_losses[-1]:.6f}")
print(f"   Total Epochs: {len(training_losses)}")

# Evaluation Functions

In [ ]:
def predict_count(
    model: torch.nn.Module,
    image_path: str,
    device: torch.device,
    img_size: int = 224
) -> Tuple[np.ndarray, float, np.ndarray]:
    """Predict crowd count for a single image."""
    model.eval()

    # Load and preprocess image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    resized_image = cv2.resize(image, (img_size, img_size))
    input_tensor = torch.from_numpy(resized_image).permute(2, 0, 1).float().unsqueeze(0) / 255.0
    input_tensor = input_tensor.to(device)

    with torch.no_grad():
        pred_density_map = model(input_tensor)

    # The predicted count is the sum of the density map
    predicted_count = pred_density_map.sum().item()

    return image, predicted_count, pred_density_map.squeeze().cpu().numpy()

def get_ground_truth_count(
    image_name: str, 
    label_dir: str
) -> int:
    """Get ground truth count from JSON label file."""
    label_file = image_name.replace('.jpg', '.json')
    label_path = os.path.join(label_dir, label_file)
    if not os.path.exists(label_path):
        # For test set, we might not have labels. We'll use train labels for MAE calculation.
        return 0
    with open(label_path, 'r') as f:
        data = json.load(f)
    return data['human_num']

def visualize_detections(
    original_image: np.ndarray,
    pred_map: np.ndarray,
    pred_count: float,
    true_count: int,
    img_name: str,
    downscale_factor: int = 16,
    threshold_scale: float = 2.0
) -> None:
    """
    Finds peaks in the density map and draws circles on the original image.
    """
    # Find local maxima in the density map
    footprint = np.ones((3, 3))
    local_max = maximum_filter(pred_map, footprint=footprint) == pred_map

    # Apply a threshold to filter out weak peaks
    threshold = pred_map.mean() * threshold_scale
    peaks = (pred_map > threshold) & local_max

    # Get coordinates of the peaks
    peak_coords = np.argwhere(peaks) # (row, col) format

    # --- Visualization ---
    plt.figure(figsize=(10, 8))

    # Draw circles on the original image
    img_with_circles = original_image.copy()
    orig_h, orig_w = img_with_circles.shape[:2]

    for y_map, x_map in peak_coords:
        # Scale coordinates from map size back to original image size
        x_orig = int((x_map + 0.5) * downscale_factor * (orig_w / 224))
        y_orig = int((y_map + 0.5) * downscale_factor * (orig_h / 224))

        # Draw a red circle
        cv2.circle(img_with_circles, (x_orig, y_orig), radius=15, color=(255, 0, 0), thickness=3)

    plt.imshow(img_with_circles)
    plt.title(f'Detections for: {img_name}\nPredicted Count: {pred_count:.2f} | Ground Truth: {true_count}', fontsize=14)
    plt.axis('off')
    plt.show()

print("✅ Evaluation functions implemented!")

# Model Evaluation

In [ ]:
print("🎯 Evaluating model on training images (to check learning)...")
print("=" * 50)

evaluation_results = []
sample_images_for_eval = train_dataset.image_files[:10] # Use first 10 training images for eval

for img_name in sample_images_for_eval:
    img_path = os.path.join(config['img_dir'], img_name)

    # Get ground truth
    true_count = get_ground_truth_count(img_name, config['label_dir'])

    # Get prediction
    original_image, pred_count, pred_map = predict_count(model, img_path, device, config['img_size'])

    evaluation_results.append({
        'image_name': img_name,
        'true_count': true_count,
        'pred_count': pred_count
    })

    # Visualize first 3 images
    if len(evaluation_results) <= 3:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
        ax1.imshow(original_image)
        ax1.set_title(f'Original Image: {img_name}')
        ax1.axis('off')

        im = ax2.imshow(pred_map, cmap='jet')
        ax2.set_title(f'Predicted Density Map (Count: {pred_count:.2f})\nGround Truth: {true_count}')
        ax2.axis('off')
        fig.colorbar(im, ax=ax2)
        plt.show()

# Calculate and display MAE
eval_df = pd.DataFrame(evaluation_results)
mae = (eval_df['pred_count'] - eval_df['true_count']).abs().mean()

print("\n📊 Evaluation Results on Training Sample:")
print(eval_df)
print(f"\n📈 Mean Absolute Error (MAE): {mae:.4f}")

# Test Prediction and Generate Submission

In [ ]:
# Create test dataset for inference
class TestDataset(Dataset):
    """Simple test dataset for inference."""
    
    def __init__(self, img_dir: str, img_size: int = 224) -> None:
        self.img_paths: List[str] = sorted(
            [os.path.join(img_dir, f) for f in os.listdir(img_dir) if f.endswith('.jpg')],
            key=lambda x: int(os.path.splitext(os.path.basename(x))[0])
        )
        self.img_size: int = img_size

    def __len__(self) -> int:
        return len(self.img_paths)

    def __getitem__(self, index: int) -> Tuple[str, np.ndarray]:
        img_path: str = self.img_paths[index]
        image_name: str = os.path.basename(img_path)
        return image_name, img_path

# Generate submission
print("📝 Generating submission file for the test set...")

test_images = sorted(
    [f for f in os.listdir(config['test_dir']) if f.endswith('.jpg')],
    key=lambda x: int(os.path.splitext(x)[0])
)
submission_data = []

for img_name in test_images:
    img_path = os.path.join(config['test_dir'], img_name)
    _, pred_count, _ = predict_count(model, img_path, device, config['img_size'])

    submission_data.append({
        'image_id': img_name,
        'predicted_count': int(round(pred_count))
    })

submission_df = pd.DataFrame(submission_data)
submission_csv_path = os.path.join(config['save_dir'], 'submission.csv')
submission_df.to_csv(submission_csv_path, index=False)

print(f"✅ Submission saved to {submission_csv_path}")
print("\n📋 Sample submission:")
submission_df.head(50)

# Model Saving

In [ ]:
# Model Saving for Competition Submission
print("💾 Saving trained model...")

# Save complete model (architecture + weights)
model_complete_path = os.path.join(config['save_dir'], 'density_fcn_model_complete.pth')
torch.save(model, model_complete_path)
print(f"✅ Complete model saved to: {model_complete_path}")

# Save model state dict only
model_weights_path = os.path.join(config['save_dir'], 'density_fcn_model_weights.pth')
torch.save(model.state_dict(), model_weights_path)
print(f"✅ Model weights saved to: {model_weights_path}")

print("\n✅ Training and evaluation complete!")
print(f"🎯 Model saved at: {config['save_dir']}")
print("📊 Evaluation results and visualizations have been generated above.")